USE SKLEARN DIFFERENT MODLELS AND FIND ONE/ONES WITH BEST PERFORMANCE

In [ ]:
import numpy as np
import pandas as pd
import math 
import sys
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline


In [ ]:

train_inputs = pd.read_csv("train.csv")
train_inputs = np.array(train_inputs)[:,:-1] 

train_labels = pd.read_csv("train_result.csv")
train_labels = np.array(train_labels)[:,1] 

test_inputs = pd.read_csv("test.csv")
test_inputs = np.array(test_inputs)[:,:-1] 



-Each Image is 1568 feature-long. 
-Each image contains 2 digits.
    - Representing a 56x28 pixel image 

-Each digit is a total of 784 pixel (i.e. 1568/2)
    - Representing a 28x28 pixel image 

    

In [ ]:
# Try displaying one image 
sampleImage = train_inputs[1,:]
print(sampleImage.shape)
fig = plt.figure
image = np.reshape(sampleImage, (28,56))
plt.imshow(image, cmap='gray_r')
plt.show()

Overview of label data. 
Labels go from 0 to 18 and we observe a bell-shaped (gaussian-like) distribution.

In [ ]:
# Class labels summary

from collections import Counter
print(train_labels.shape)
print(np.unique(train_labels))
c = Counter(train_labels)
plt.bar(c.keys(), c.values())
plt.show()

In [ ]:
# Stackoverflow suggested solution for one-hot encoding - Source : https://stackoverflow.com/questions/38592324/one-hot-encoding-using-numpy
#@staticmethod
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

Y_onehot_train = get_one_hot(np.int_(train_labels), np.unique(train_labels).size)

In [ ]:
#Consusion matrix for calculating errors

def conf_matrix(testlabels, predlabels):

    n_classes = 19 
    matrix = np.zeros((n_classes,n_classes))
    

    for (test, pred) in zip(testlabels, predlabels):
        matrix[int(test)-1,int(pred)-1]+=1
    return matrix



In [ ]:
#Splitting train to use data as validation data
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(train_inputs, train_labels, test_size=0.30)

## SVM

In [ ]:
import sklearn.svm as svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

In [ ]:
svm1 = svm.SVC(kernel="rbf", C=5, gamma = 0.05)
svm1.fit(train_inputs, train_labels)

In [ ]:
svm2 = svm.SVC(kernel = 'poly')
svm2.fit(X_train,y_train)

In [ ]:
svm3 = svm.SVC(kernel="sigmoid")
svm3.fit(X_train,y_train)

In [ ]:
classes_pred_svm1 = svm1.predict(X_test)
classes_pred_svm2 = svm2.predict(X_test)
classes_pred_svm3 = svm3.predict(X_test)

In [ ]:
def calculateError(classes_pred):
    
    confmat = conf_matrix(y_test, classes_pred)
    sum_preds = np.sum(confmat)
    sum_correct = np.sum(np.diag(confmat))
    print("The test error is ", round(100 * (1.0 - (float(sum_correct) / float(sum_preds))), 2), "%")


calculateError(classes_pred_svm1)
calculateError(classes_pred_svm2)
calculateError(classes_pred_svm3)

In order : 

svm1 = svm.SVC(kernel="rbf", C=5, gamma = 0.05)
svm2 = svm.SVC(kernel="poly")
svm3 = svm.SVC(kernel="sigmoid")



The test error is  55.43 %
The test error is  26.04 %
The test error is  93.5 %

In [ ]:
# with Standardisation
svm1_std = make_pipeline(StandardScaler(), svm.SVC(kernel="rbf", C=5, gamma = 0.05))
svm1_std.fit(X_train, y_train)

In [ ]:
svm2_std = make_pipeline(StandardScaler(), svm.SVC(kernel="poly"))
svm2_std.fit(X_train,y_train)

In [ ]:
# svm3_std = make_pipeline(StandardScaler(), svm.SVC(kernel="sigmoid"))
# svm3_std.fit(X_train,y_train)

In [ ]:
classes_pred_svm1_std = svm1_std.predict(X_test)
classes_pred_svm2_std = svm2_std.predict(X_test)
# classes_pred_svm3_std = svm3_std.predict(X_test)

On voit que la standardisation n'a pas eu un effet positif.

In [ ]:
print("For Standardized data - SVM with kernel rbf") ; calculateError(classes_pred_svm1_std)
print("For Standardized data - SVM with kernel poly") ;calculateError(classes_pred_svm2_std)
# print("For SVM with kernel sigmoid \n") ;calculateError(classes_pred_svm3_std)

## NAIVES BAYES 

Sans aucune forme de pré-traitement, le classifieur Naive Bayes obtient un score de 7,39% seulement sur D_test.

In [ ]:
gaussianClassifier = GaussianNB()
gaussianClassifier.fit(X_train, y_train)
pred_test = gaussianClassifier.predict(X_test)
print(accuracy_score(y_test, pred_test)*100)

Pour réduire le nombre de features que doit apprendre le Naive Bayes, on va faire une PCA en premier. 

On va essayer de trouver le parametre optimal d'une PCA, suivi d'un classifieur Bayes naif.
Pour PCA, il s'agit de savoir combien de composantes principales garder, pour generer la meilleure prediction par Naive Bayes.

On peut regrouper la procédure (PCA ensuite Bayes) dans un pipeline sklearn.

In [ ]:
# Fit to training data and predict on test data (PCA + Naive Bayes)
PCA_NaiveBayes = make_pipeline(PCA(n_components=25), GaussianNB())
PCA_NaiveBayes.fit(X_train, y_train)
pred_test = PCA_NaiveBayes.predict(X_test)
print(accuracy_score(y_test, pred_test)*100)


In [ ]:
max_components_to_test = 250
kComponents = [k+1 for k in range(max_components_to_test)]
def findBestNumberComponentsForPCA_NaiveBayes() : 
    accuracyArr = []

    bestParameter = -1
    bestAccuracy = -1
    for k in kComponents : 
        # Fit to training data and predict on test data (PCA with k components + Naive Bayes)
        PCA_NaiveBayes = make_pipeline(PCA(n_components=k), GaussianNB())
        PCA_NaiveBayes.fit(X_train, y_train)
        pred_test = PCA_NaiveBayes.predict(X_test)
        accuracy = accuracy_score(y_test, pred_test)*100
        if k%10 ==0 : print(f'Accuracy for PCA with {k} components is : {accuracy}%')
        accuracyArr.append(accuracy)
        if accuracy>bestAccuracy :
            bestAccuracy = accuracy
            bestParameter = k

    return bestParameter, bestAccuracy, accuracyArr

In [ ]:
bestParameter, bestAccuracy, accuracyArr = findBestNumberComponentsForPCA_NaiveBayes()

On constate que la précision du classifieur Naive Bayes va en augmentant jusqu'à un certain seuil, au delà duquel la précision du classifieur diminue graduellement. 

Le nombre optimal de Composantes Principales à utiliser ici avec Naive Bayes est 64, et cela permet un score de 23.56%.

In [ ]:
print(f'Number of Principal Components that generated the best prediction with Naive Bayes : {bestParameter}')
print(f'Accuracy : {bestAccuracy}')

plt.plot(kComponents, accuracyArr)
plt.title('Accuracy % on randomized test data when using Naives Bayes with the K Principal components of training data')
plt.xlabel('Number of components used (post-PCA)')
plt.ylabel('Score on test data (% accuracy)')
plt.show()

## KNN

In [ ]:
import random

In [ ]:
def minkowski_mat(x, Y, p=2):
    return (np.sum((np.abs(x - Y)) ** p, axis=1)) ** (1.0 / p)

In [ ]:
#Inspired by labo2
class NeighborhoodClassifier:
    def __init__(self, dist_func=minkowski_mat, k=1):

        self.dist_func = dist_func
        self.k = k

    # The train function for knn 
    def train(self, train_inputs, train_labels):
        self.train_inputs = train_inputs
        self.train_labels = train_labels
        self.n_classes = len(np.unique(train_labels))

    # The prediction function takes as input test_data and returns an array containing the predicted classes. 
    def compute_predictions(self, test_data):
        # Initialization of the count matrix and the predicted classes array
        num_test = test_data.shape[0] #nombre de lignes de test
        counts = np.ones((num_test, self.n_classes)) 
        classes_pred = np.zeros(num_test) 
        # For each test datapoint
        
        for (i, ex) in enumerate(test_data):
  
          
            # i is the row index
            # ex is the i'th row

            # Find the distances to each training set point using dist_func
            
          distances = self.dist_func(ex,self.train_inputs)


          labelToCount = np.array([])
          
          labeltoCount = [self.train_labels[int(x)] for x in np.argsort(distances)[:self.k]]

            
            # Calculate the number of neighbors belonging to each class and write them in counts[i,:]
            
          for label in range(self.n_classes):
            counts[i,label] = labeltoCount.count(label+1)

           
            # From the counts matrix, define classes_pred[i]
            

          classes_pred[i] = int(np.argmax(counts[i]) + 1) 
        return classes_pred.astype('int32')

With validation set

In [ ]:
# load mnist


# Number of classes
n_classes = 19
# Size of training set
n_train = 40000

data = np.array(train_inputs)[:50000,:] #n_train and the data range can be changed 
labels = np.array(train_labels)[:50000] 


# The columns (features) on which to train our model

train_cols = [x for x in range(56*28)]


# Comment to have random (non-deterministic) results
random.seed(3395)
# Randomly choose indexes for the train and test dataset
inds = list(range(data.shape[0]))
random.shuffle(inds)
train_inds = inds[:n_train]
test_inds = inds[n_train:]

# Split the data into both sets
train_set = data[train_inds, :]
train_set = train_set[:, train_cols] 
valid_set = data[test_inds, :]
valid_set = valid_set[:, train_cols]

# Separate the test set into inputs and labels
valid_inputsKNN = valid_set
valid_labelsKNN = labels[test_inds]
train_inputsKNN = train_set
train_labelsKNN = labels[train_inds]



On test set

In [ ]:
# load mnist


# Number of classes
n_classes = 19
# Size of training set
n_train=50000

 
# The columns (features) on which to train our model
# For gridplot to work, len(train_cols) should be 2
train_cols = [x for x in range(56*28)]


# Comment to have random (non-deterministic) results
random.seed(3395)
# Randomly choose indexes for the train and test dataset
inds = list(range(data.shape[0]))
random.shuffle(inds)
train_inds = inds[:n_train]


# Split the data into both sets
train_set = train_inputs[train_inds, :]
test_set = test_inputs


# Separate the test set into inputs and labels
test_inputsKNN = test_set
train_inputsKNN = train_set
train_labelsKNN = train_labels[train_inds]

In [ ]:
# Number of neighbors (k) for knn
k = 9

# Create the classifiers
knn = NeighborhoodClassifier(dist_func=minkowski_mat, k=k)


# We train the models
knn.train(train_inputsKNN, train_labelsKNN)


# We get predictions
classes_pred_knn = knn.compute_predictions(test_inputsKNN)


In [ ]:
def show_results(model, classes_pred):
    # Confusion Matrix
    confmat = conf_matrix(valid_labelsKNN, classes_pred)
    print('The confusion matrix is:')

    # Test error
    sum_preds = np.sum(confmat)
    sum_correct = np.sum(np.diag(confmat))
    print("The test error is ", round(100 * (1.0 - (float(sum_correct) / float(sum_preds))), 2), "%")

show_results(knn, classes_pred_knn)

Optimisation des parametres

In [ ]:
def get_test_error(k):
   
    knn = NeighborhoodClassifier(dist_func=minkowski_mat, k=k)
    knn.train(train_inputs, train_labels)
    classes_pred_knn = knn.compute_predictions(test_inputs)
    confmat = conf_matrix(test_labels, classes_pred_knn)
    print(k)
    return 1.0 - (float(np.sum(np.diag(confmat))) / float(np.sum(confmat)))
    

In [ ]:
plt.plot(range(1, 30), [get_test_error(k) for k in range(1, 30)], label='test error')
plt.legend()
plt.xlabel('number of neighbors')
plt.show()

# Neural Network


https://www.linkedin.com/pulse/choosing-number-hidden-layers-neurons-neural-networks-sachdev/

https://victorzhou.com/blog/keras-cnn-tutorial/

https://keras.io/examples/vision/mnist_convnet/ # 99% accuracy

In [ ]:
from keras.datasets import mnist     
from keras.models import Sequential  

from keras.layers.core import Dense, Dropout, Activation 
from keras.utils import np_utils                         
from sklearn.model_selection import train_test_split 

In [ ]:
#Sur Valid set
X_train, X_test, y_train, y_test = train_test_split(train_inputs, train_labels, test_size=0.30)

#Sur test set
"""
X_train = train_inputs
y_train = train_labels
X_test = test_inputs"""

Convert y to one-hot

In [ ]:
no_classes = 19

Y_train = np_utils.to_categorical(y_train, no_classes)
Y_test = np_utils.to_categorical(y_test, no_classes)

In [ ]:
model = Sequential()

First Hidden Layer

In [ ]:
model.add(Dense(800, input_shape=(1568,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

Second Hidden Layer

In [ ]:
model.add(Dense(172))
model.add(Activation('relu'))
model.add(Dropout(0.2))

Third Hidden Layer

In [ ]:
model.add(Dense(170))
model.add(Activation('relu'))
model.add(Dropout(0.2))

Output Layer

In [ ]:
model.add(Dense(19))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Compiling the model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train,
          batch_size=128, epochs=20,
          verbose=1)

Evaluate the model

In [ ]:
score = model.evaluate(X_test, Y_test)
print(model.predict(X_test))
print('Test accuracy:', score[1])

In [ ]:
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

In [ ]:
results = model.predict(X_test)

In [ ]:
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Class")
submission = pd.concat([pd.Series(np.arange(0,len(X_test) ).astype(int),name = "Index"),results],axis = 1)
submission.to_csv("submission.csv",index=False)